# Maestría en "Analítica de datos"
---
Nombre: Luis Fernando Castellanos Guarin
Código: XXXX
Fecha: 2025 agosto 28
---
Descripción: este libro trabaja ejercicios de "procesamiento en paralelo"

# 1.Cargar librerias

In [7]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
url_forder_csv="/content/drive/MyDrive/UniversidadCentral/Maestría_en_Analítica_de_Datos/Bigdata/DataSets_TMP/tutelas_con_providencias"

# 2. carga/obtención de información

In [5]:
listado_archivos= [f for f in os.listdir(url_forder_csv) if f.endswith(".csv")]
print(listado_archivos)

['2025.csv', '2024.csv', '2023.csv', '2021.csv', '2022.csv']


In [16]:
columnas_nombre=[
    "clasificacion",
    "num_expediente",
    "fecha_Radica",
    "demandante",
    "demandado",
    "primera_instancia",
    "segunda_instancia",
    "num_23Digitos",
    "providencia",
    "fechaSentencia"
]
df_tutelas=pd.DataFrame(columns=columnas_nombre)
display (df_tutelas)

,clasificacion,num_expediente,fecha_Radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fechaSentencia


##2.a procesamiento secuencial (uno tras otro)

In [ ]:
df_list=[]  #lista de df resultante de los datos de cada uno de los CSV
for archivo_nombre in listado_archivos:
  ruta_archivo= os.path.join(url_forder_csv,archivo_nombre) #ruta completa del archivo
  print(ruta_archivo)

  df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')
  #agregarle nombre de columnas al df creado
  df_temporal.columns=columnas_nombre

  if (len(df_temporal.columns)==len(columnas_nombre)):
    df_list.append(df_temporal)
    print(f" archivo {archivo_nombre} cargada exitosamente")
  else:
    print(f"archivo no trabajado {archivo_nombre} no tiene 10 columnas")
#--------una vez termine de cargar todos los CSV combinelos en el dataframa df_tutelas
if df_list:
  df_tutelas= pd.concat(df_list,ignore_index=True)
  # reemplazar las palabras "NULL" por none
  df_tutelas.replace("NULL",None,inplace=True)
  display(df_tutelas.head())


##2.b procesamiento en paralelo (todos al tiempo)

In [13]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [17]:
def cargar_data_desde_archivo_csv(ruta_archivo):
  try:
      df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')
      #agregarle nombre de columnas al df creado
      df_temporal.columns=columnas_nombre
      if (len(df_temporal.columns)==len(columnas_nombre)):
        print(f" archivo {os.path.basename(ruta_archivo)} cargada exitosamente")
        return df_temporal
      else:
        print(f"archivo no trabajado {os.path.basename(ruta_archivo)} no tiene 10 columnas")
        return None

  except Exception as e:
      print(f"Error al procesar el archivo {os.path.basename(ruta_archivo)}: {str(e)}")
      return None

In [18]:
df_list=[]  #lista de df resultante de los datos de cada uno de los CSV
with ThreadPoolExecutor() as executor:
  future_to_file= {executor.submit(cargar_data_desde_archivo_csv,os.path.join(url_forder_csv,archivo_nombre)):archivo_nombre for archivo_nombre in listado_archivos}
  for future in as_completed(future_to_file):
    df=future.result()
    if df is not None:
      df_list.append(df)
#--------una vez termine de cargar todos los CSV combinelos en el dataframa df_tutelas
if df_list:
  df_tutelas= pd.concat(df_list,ignore_index=True)
  # reemplazar las palabras "NULL" por none
  df_tutelas.replace("NULL",None,inplace=True)
  display(df_tutelas.head())


 archivo 2021.csv cargada exitosamente


/tmp/ipython-input-3231973599.py:3: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temporal = pd.read_csv(ruta_archivo,sep=';',header=None,encoding='latin-1',on_bad_lines='skip')


 archivo 2025.csv cargada exitosamente
 archivo 2022.csv cargada exitosamente
 archivo 2023.csv cargada exitosamente
 archivo 2024.csv cargada exitosamente


,clasificacion,num_expediente,fecha_Radica,demandante,demandado,primera_instancia,segunda_instancia,num_23Digitos,providencia,fechaSentencia
0,ï»¿Acciones de Tutela,T8531729,2021-12-16 00:00:00,VAZQUEZ CUADROS BAYRON EMILIO Y OTROS EN REPRE...,SOCIEDAD DE ACTIVOS ESPECIALES S.A.E. S.A.S.,"BOGOTA,CUNDINAMARCA, TRIBUNAL SUPERIOR DE BOGO...",--,--,--,NaN
1,Acciones de Tutela,T8531726,2021-12-16 00:00:00,ESCORCIA JORGE ARMANDO,ALCALDIA MUNICIPAL DE SANTA LUCIA,"SANTA LUCIA,ATLANTICO, JUZGADO 1 PROMISCUO MUN...","SABANALARGA,ATLANTICO, JUZGADO 2 PROMISCUO DEL...",--,--,NaN
2,Acciones de Tutela,T8531728,2021-12-16 00:00:00,ARCINIEGAS MEDRANO CLEMENCIA,COLPENCIONES Y OTRO,"BARRANQUILLA,ATLANTICO, JUZGADO 11 LABORAL DEL...","BARRANQUILLA,ATLANTICO, TRIBUNAL SUPERIOR SALA...",--,T-274/22,2022-07-28 00:00:00.000
3,Acciones de Tutela,T8531723,2021-12-16 00:00:00,SERRANO ROJAS CAMILO ANDRES,SECRETARIA DE MOVILIDAD DE CAQUEZA,"CAQUEZA,CUNDINAMARCA, JUZGADO 1 PROMISCUO MUNI...",--,--,--,NaN
4,Acciones de Tutela,T8531727,2021-12-16 00:00:00,GOMEZ QUINTERO EDILSA MARIA,CAJACOPI EPS Y OTRO,"CURUMANI,CESAR, JUZGADO PROMISCUO MUNICIPAL","CHIRIGUANA,CESAR, JUZGADO PENAL DEL CIRCUITO",--,--,NaN
